In [4]:
import faiss
import numpy as np
import pandas as pd
import logging
from datetime import datetime
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer
import time
# 로거 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('embedding_log.log'),
        logging.StreamHandler()
    ]
)

class LocalEmbeddingGenerator:
    def __init__(self, model_name="BAAI/bge-m3", device='cuda', batch_size=32):
        self.model = SentenceTransformer(model_name, device=device)
        self.batch_size = batch_size
        self.device = device
        self.total_processed = 0
        
        if not torch.cuda.is_available():
            logging.warning("CUDA is not available. Using CPU instead.")
            self.device = 'cpu'

    def generate_embeddings(self, texts):
        try:
            embeddings = self.model.encode(texts, 
                                         batch_size=self.batch_size,
                                         convert_to_numpy=True,
                                         normalize_embeddings=True)
            self.total_processed += len(texts)
            return embeddings
        except Exception as e:
            logging.error(f"Error generating embeddings: {str(e)}")
            raise

if __name__ == '__main__':
    start_total = time.time()
    logger = logging.getLogger(__name__)
    
    try:
        logger.info("Loading dataset...")

        # Excel 파일 로딩
        data = pd.read_excel("data_with_page.xlsx")
        grouped = data.groupby("page")["text"].apply(lambda x: " ".join(map(str, x))).reset_index()

        texts_by_page = grouped["text"].tolist()
        page_ids = grouped["page"].tolist()
        total_records = len(texts_by_page)
        
        embedder = LocalEmbeddingGenerator(device='cuda')
        
        embeddings = []
        progress_bar = tqdm(total=total_records, desc="Generating Embeddings", unit="page")

        for i in range(0, total_records, embedder.batch_size):
            batch_texts = texts_by_page[i:i+embedder.batch_size]
            
            try:
                batch_embeddings = embedder.generate_embeddings(batch_texts)
                embeddings.extend(batch_embeddings)
                
                progress_bar.update(len(batch_texts))
                progress_bar.set_postfix({
                    "Device": embedder.device.upper(),
                    "Batch Size": embedder.batch_size,
                    "Processed": embedder.total_processed
                })
                
            except Exception as e:
                logger.error(f"Error processing batch {i//embedder.batch_size}: {str(e)}")
                continue

        logger.info("Creating FAISS index...")
        dimension = embeddings[0].shape[0]
        embeddings_array = np.array(embeddings, dtype='float32')
        
        index = faiss.IndexFlatIP(dimension)
        index.add(embeddings_array)
        faiss.write_index(index, "vector_store_1.index")

        total_time = time.time() - start_total
        logger.info(f"""
        ===== Processing Complete =====
        Total Records: {total_records}
        Successful Embeddings: {len(embeddings)}
        Total Time: {total_time:.2f} seconds
        Embedding Speed: {total_records/total_time:.2f} rec/sec
        Device Used: {embedder.device.upper()}
        """)

    except Exception as e:
        logger.error(f"Fatal error: {str(e)}", exc_info=True)
        raise

2025-07-09 02:21:06,987 - INFO - Loading dataset...
2025-07-09 02:21:07,045 - INFO - Load pretrained SentenceTransformer: BAAI/bge-m3
Generating Embeddings: 100%|██████████| 12/12 [00:56<00:00,  4.72s/page, Device=CUDA, Batch Size=32, Processed=12]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-07-09 02:21:18,249 - INFO - Creating FAISS index...
2025-07-09 02:21:18,259 - INFO - 
        ===== Processing Complete =====
        Total Records: 12
        Successful Embeddings: 12
        Total Time: 11.27 seconds
        Embedding Speed: 1.06 rec/sec
        Device Used: CUDA
        


In [ ]:
#슬라이드 변경 감지 코드 1차 코드
import cv2
import numpy as np

def mse(img1, img2):
    return np.mean((img1.astype("float") - img2.astype("float")) ** 2)

cap = cv2.VideoCapture("test1.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * 1)  # 1초 간격
slide_times = []
last_slide = None
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (320, 240))  # 계산량 줄이기

        if last_slide is not None:
            diff = mse(gray, last_slide)
            if diff > 500:  # 임계값 (조정 가능)
                current_time = frame_count / fps
                print(f"[{frame_count/fps//60:.0f}분{frame_count/fps%60:.0f}초] 이미지 차이: {diff}")
                slide_times.append(current_time)

        last_slide = gray

    frame_count += 1

cap.release()


[0분31초] 이미지 차이: 4403.600520833334
[2분57초] 이미지 차이: 1718.61578125
[5분57초] 이미지 차이: 500.92506510416666
[6분39초] 이미지 차이: 1503.1039973958334
[7분38초] 이미지 차이: 1269.2990885416666


In [2]:
# pdf2image를 이용한 PDF 페이지 이미지 추출
import fitz 
from PIL import Image

doc = fitz.open("test1.pdf")
images = []

for i in range(len(doc)):
    page = doc[i]
    pix = page.get_pixmap(dpi=300)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    images.append(img)

# 첫 페이지 이미지 보기
#images[0].show()
#images[0].save("first_page_image.png")

In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.nn.functional import cosine_similarity
import fitz  # PyMuPDF
import io
import cv2
import numpy as np
import csv

print("🔧 모델 로딩 중...")
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1]) 
model.eval()
print("✅ 모델 로딩 완료")

# PDF 로드
print("📄 PDF 파일 로드 중...")
doc = fitz.open("test1.pdf")
print(f"✅ PDF 로드 완료 - 총 {len(doc)} 페이지")

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 로그 파일 설정
output_path = "slide_similarity_log.csv"
csv_file = open(output_path, mode="w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["seconds", "page", "similarity"])
print(f"📝 CSV 로그 파일 초기화 완료: {output_path}")

# MSE 유사도 측정 함수
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

# 프레임 → feature 추출
def extract_feature(image):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image).convert("RGB")
    else:
        image = image.convert("RGB")
    tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        feature = model(tensor).squeeze()
    return feature

# 영상 파일 로딩
print("🎞️ 영상 로딩 중...")
cap = cv2.VideoCapture("test1.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"✅ 영상 로딩 완료 - FPS: {fps:.2f}")
frame_interval = int(fps * 1)

# 상태 변수들
last_slide = None
last_best_page = 0
prev_sim = None
frame_count = 0

# 하이퍼파라미터
mse_threshold = 500
sim_drop_threshold = 0.01
max_search_range = 10

print("🚀 분석 시작")

# 메인 루프
while True:
    ret, frame = cap.read()
    if not ret:
        print("🎬 영상 끝")
        break

    frame_time = frame_count / fps

    if frame_count % frame_interval == 0:
        #print(f"\n🧭 프레임 {frame_count} (시간: {frame_time:.2f}초) 분석 중...")
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (320, 240))

        if last_slide is None:
            print("🆕 첫 슬라이드로 설정")
            last_slide = gray
        else:
            diff = mse(gray, last_slide)
            #print(f"🔍 MSE 차이: {diff:.2f}")

            if diff > mse_threshold:
                print("📈 슬라이드 변경 감지됨 → 탐색 시작")
                frame_sim_results = []

                for offset in range(-int(2 * fps), int(2 * fps) + 1, frame_interval):
                    pos = frame_count + offset
                    if pos < 0 or pos >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
                        continue

                    cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
                    ret2, temp_frame = cap.read()
                    if not ret2:
                        continue

                    frame_rgb = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2RGB)
                    feat1 = extract_feature(frame_rgb)

                    # 기본 후보 페이지 설정
                    candidate_range = [-2, -1, 0, 1, 2]
                    candidates = [last_best_page + i for i in candidate_range if 0 <= last_best_page + i < len(doc)]

                    best_page = last_best_page
                    max_sim = -1

                    print(f"🔎 1차 탐색: 후보 페이지 {candidates}")
                    for i in candidates:
                        page = doc[i]
                        pix = page.get_pixmap(dpi=300)
                        img_bytes = pix.tobytes("ppm")
                        feat2 = extract_feature(Image.open(io.BytesIO(img_bytes)))
                        sim = cosine_similarity(feat1.unsqueeze(0), feat2.unsqueeze(0)).item()

                        print(f"    페이지 {i} 유사도: {sim:.4f}")
                        if sim > max_sim:
                            max_sim = sim
                            best_page = i
                            best_feat2 = feat2

                    # 유사도 급락 시 재탐색
                    if prev_sim is not None and (prev_sim - max_sim) >= sim_drop_threshold:
                        print(f"⚠️ 유사도 하락 감지 (이전: {prev_sim:.4f} → 현재: {max_sim:.4f}) → 2차 탐색")
                        expanded_range = list(range(-5, 6))
                        expanded_candidates = [last_best_page + i for i in expanded_range if 0 <= last_best_page + i < len(doc)]

                        print(f"🔍 2차 탐색: 후보 페이지 {expanded_candidates}")
                        for i in expanded_candidates:
                            page = doc[i]
                            pix = page.get_pixmap(dpi=300)
                            img_bytes = pix.tobytes("ppm")
                            feat2 = extract_feature(Image.open(io.BytesIO(img_bytes)))
                            sim = cosine_similarity(feat1.unsqueeze(0), feat2.unsqueeze(0)).item()

                            print(f"    [2차] 페이지 {i} 유사도: {sim:.4f}")
                            if sim > max_sim:
                                max_sim = sim
                                best_page = i
                                best_feat2 = feat2

                    result_time = pos / fps
                    minutes = int(result_time // 60)
                    seconds = int(result_time % 60)
                    print(f"✅ [결과] {minutes}분 {seconds}초: 페이지 {best_page} / 유사도 {max_sim:.4f}")
                    csv_writer.writerow([round(result_time, 2), best_page, round(max_sim, 4)])
                    frame_sim_results.append((max_sim, best_page, result_time, gray))

                if frame_sim_results:
                    best_result = max(frame_sim_results, key=lambda x: x[0])
                    prev_sim = best_result[0]
                    last_best_page = best_result[1]
                    last_slide = best_result[3]
                    print(f"📌 현재 상태 갱신 → 페이지 {last_best_page}, 유사도 {prev_sim:.4f}")

    frame_count += 1

cap.release()
csv_file.close()
print("✅ 모든 작업 완료. 로그 저장됨.")


🔧 모델 로딩 중...
✅ 모델 로딩 완료
📄 PDF 파일 로드 중...
✅ PDF 로드 완료 - 총 33 페이지
📝 CSV 로그 파일 초기화 완료: slide_similarity_log.csv
🎞️ 영상 로딩 중...
✅ 영상 로딩 완료 - FPS: 30.00
🚀 분석 시작
🆕 첫 슬라이드로 설정
📈 슬라이드 변경 감지됨 → 탐색 시작
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.9955
    페이지 1 유사도: 0.8803
    페이지 2 유사도: 0.7426
✅ [결과] 0분 29초: 페이지 0 / 유사도 0.9955
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.9955
    페이지 1 유사도: 0.8803
    페이지 2 유사도: 0.7426
✅ [결과] 0분 30초: 페이지 0 / 유사도 0.9955
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 31초: 페이지 1 / 유사도 0.9950
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 32초: 페이지 1 / 유사도 0.9950
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 33초: 페이지 1 / 유사도 0.9950
📌 현재 상태 갱신 → 페이지 0, 유사도 0.9955
📈 슬라이드 변경 감지됨 → 탐색 시작
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8836
    페이지 1 유사도: 0.9941
    페이지 2 유사도: 0.8560
✅ [결과] 2분 53초: 페이지 1 / 유사도 0.9941
🔎

KeyboardInterrupt: 

In [1]:
import pandas as pd
import math

# 1. 데이터 로드
log_df = pd.read_csv("slide_similarity_log.csv")   # seconds, page
data_df = pd.read_excel("data.xlsx")               # end 열 포함

# 2. end를 올림하여 seconds로 변환
data_df["rounded_sec"] = data_df["end"].apply(lambda x: math.ceil(x))

# 3. 매핑: seconds → page
seconds_to_page = dict(zip(log_df["seconds"].astype(int), log_df["page"]))

# 4. page 열 추가 (없으면 NaN)
data_df["page"] = data_df["rounded_sec"].map(seconds_to_page)

# 5. 결과 저장
data_df.to_excel("data_with_page.xlsx", index=False)

print("✅ 저장 완료: data_with_page.xlsx")


✅ 저장 완료: data_with_page.xlsx
